In [ ]:
import numpy as np
import galois

# Create a simple Point class to represent the affine points.
from collections import namedtuple
Point = namedtuple("Point", "x y")

# The point at infinity (origin for the group law).
O = 'Origin'

def valid(P):
    """
    Determine whether we have a valid representation of a point
    on our curve.  We assume that the x and y coordinates
    are always reduced modulo p, so that we can compare
    two points for equality with a simple ==.
    """
    if P == O:
        return True
    else:
        return ((P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p)

    
    
# Defining Elliptic Curve
a = 0
b = 3
p = 101
#

def inv_mod_p(x):
    """
    Compute an inverse for x modulo p, assuming that x
    is not divisible by p.
    """
    if x % p == 0:
        raise ZeroDivisionError("Impossible inverse")
    return pow(x, p-2, p)

def ec_inv(P):
    """
    Inverse of the point P on the elliptic curve y^2 = x^3 + ax + b.
    """
    if P == O:
        return P
    return Point(P.x, (-P.y)%p)

def ec_add(P, Q):
    """
    Sum of the points P and Q on the elliptic curve y^2 = x^3 + ax + b.
    """
    if not (valid(P) and valid(Q)):
        raise ValueError("Invalid inputs")

    # Deal with the special cases where either P, Q, or P + Q is
    # the origin.
    if P == O:
        result = Q
    elif Q == O:
        result = P
    elif Q == ec_inv(P):
        result = O
    else:
        # Cases not involving the origin.
        if P == Q:
            dydx = (3 * P.x**2 + a) * inv_mod_p(2 * P.y)
        else:
            dydx = (Q.y - P.y) * inv_mod_p(Q.x - P.x)
        x = (dydx**2 - P.x - Q.x) % p
        y = (dydx * (P.x - x) - P.y) % p
        result = Point(x, y)

    # The above computations *should* have given us another point
    # on the curve.
    assert valid(result)
    return result

def addition(n,g):
    """
    Given a element g generates the cyclic subgroup using bruteforce. 
    """
    Gen = g
    for i in range(1,n):
        g = ec_add(Gen,g)
    return g




# Summary

This is a toy implementation of Plonk that follows the plonk-by-hand series written by Metastate. It is meant to be a interactive example of their notes. As such it does not really introduce anything new. However, it goes over what a prover and verifier would need to compute in a PLONK snark. 


Some reflection in this endeavor:

- Implementating production level Snarks is a monumental task. Even a prover and verifier for a simple circuit required a good bit of coding. In this simple example there are still missing components:
    - Field Extensions for Pairing Map
    - Incomplete KZG commitment scheme
    - This implementation is not non-interactive
- For pedagogy, there are two ways to approaches SNARKs:
    - As a "cryptographer" - Understand enough theory to grok the scheme
    - As a "Developer" - Understand how to implement and build more complex circuits
    
Both require a good bit of time and background knowledge. Overall this was an interesting learning experience. Maybe not the best approach. However there are a few things to finalize and add overtime:

- Define the necessary reason for each step of the Proof
- Review the permutation check in detail. 

Unfortunately, I do no believe this computation will be enlightening to a reader. So instead here are some more illuminating articles that cover PLONK in more detail:

- [Anatomy of Proof Generation](https://scroll.io/blog/proofGeneration#heading-22) - Scroll Team
- [PLONK Paper](https://eprint.iacr.org/2019/953.pdf) - Gabizon, Williamson, Ciobotaru
- [PLONK by HAND](https://research.metastate.dev/plonk-by-hand-part-2-the-proof/) - Metastate Team
- [Understanding PLONK](https://vitalik.ca/general/2019/09/22/plonk.html) - Buterin

# Run Trusted Setup

- A circuit with n gates requires a structured reference string (SRS) with at least n+5 elements 
- Circuit in example has 4 gates therefore we need 9 elements


In [ ]:
import random 

def randomset(order,power):
    hold = []
    for i in range(0,order):
        hold.append(random.randrange(1,power+1))
    return hold

# Use for 'true' randomness
randomSet= randomset(6,17)

# Use to follow article
randomness = [1,2,4,8,16,15,13]


# Generates Points for Generater G1 in pairing
cermony = [addition(i,Point(1,2)) for i in randomness]

# Generate Points for Generator G2 in pairing

# Convert Equation into Circuit representation

- Convert Polynomial into directed graph such that operations are preserved.
- We will do this by example. 
- In the current example: $3^2 + 4^2 = 5^2$
## 1. Rewrite as Circuit

In [ ]:
# Convert Equation into Circuit representation

#  
#-
#-                                   
#-                 |
#-         --------+--------        --*--
#-         |               |       |    | 
#-     ----*-----      ----*---    5----
#-     |        |      |.     |
#-     3---->----      4--->---



## 2. Represent Circuit relations 



- The first gate is a multiplication gate represented by the closed 
loop that contains 3. This gives the following relationship:
$$3 * 3 = 9$$
$$x_1*x_1=x2$$

- The second gate is a multiplication gate represented by the closed 
loop that contains 4. This gives the following relationship:
$$4 * 4 = 16$$
$$x_3*x_3=x4$$

- The third gate is a multiplication gate represented by the closed 
loop that contains 5. This gives the following relationship:
$$5 * 5 = 25$$
$$x_5*x_5=x6$$

- The fourth gate is an addition gate represented the result of the connected multiplication gates at the addition vertex. This gives the following relationship:
$$9 * 16 = 25$$
$$x_2*x_4=x6$$

- We can represent these relationships as matrix:


\begin{matrix}
Column| & A & B & C\\
Gate 1| & 3 & 3 & 9\\
Gate 2| &4 & 4 & 16\\
Gate 3| &5 & 5 & 25\\
Gate 4| &9 & 16& 25\\
\end{matrix}

- Each gate is represented with a more generic equation:

$$q_l*a + q_r*b + q_O*c + q_m*ab + q_c =0 $$


\begin{matrix}
Column| & q_l*a & q_r*b & q_O*ab &q_m*ab &q_c&0\\
Gate 1| &0*a_1 & 0*b_1 &-1*c_1&1*a_1b_1&0&0\\
Gate 2| &0*a_2 & 0*b_2 &-1*c_2&1*a_2b_2&0&0\\
Gate 3| &0*a_3 & 0*b_3 & -1*c_3&1*a_3b_3&0&0\\
Gate 4| &1*a_4 & 1*b_4& -1*c_4&1*a_4b_4&0&0\\
\end{matrix}

- For computations we collect the column vectors:


#  Assignment Vectors 

$$A = [3,4,5,9]$$
$$B = [3,4,5,16]$$
$$C = [9,16,8,8]$$

#  Selector Vectors  



$$l = [0,0,0,1] $$
$$r = [0,0,0,1]$$
$$o = [-1,-1,-1,-1] $$
$$m = [1,1,1,0]$$
$$c = [0,0,0,0]$$


In [ ]:

####  Assignment Vectors

A = [3,4,5,9]
B = [3,4,5,16]
C = [9,16,8,8]


###   Selector Vectors  

l = [0,0,0,1] # Selector Polynomials 
r = [0,0,0,1]
o = [-1%17,-1%17,-1%17,-1%17]
m = [1,1,1,0]
c = [0,0,0,0]


# Naive Polynomial Interpolation

Given a degree n polynomial f(x), if we have n+1 points that
lay on f(x) then we can recover the equation of f(x).  In PLONK, the domain that is used for polynomial interpolation consists of roots of unity. 

The general pattern for polynomial interpolation is as follows.


- Suppose we have the polynomial where a_i are unknown:
$$f(x) = a_0 + a_1 x + ... + a_n x^n$$
- We have $n+1$ points $(x_i,y_i)$.

- We can then represent a linear system $Ax=b$ whose rows are defined by $f(x_i) = y_i$ where the variables are given by $a_i$.

- Solve for the unknown x by finding $A^{-1} b$. This results in finding the polynomial f(x). 

In [ ]:
# Given a column vector we can find a polynomial to represent it.
#
# 1. For a vector b with n entries, rewrite it as a nth degree polynomial f
# 2. Use the nth roots of unity and evaluate to form a linear system A_fx = b
# 3. Solve for x = A_f^-1x * b
# 4. cofeeicents of f is given by x  
def interpolate_polynomial(constraints,order_of_field,root):
    GF = galois.GF(order_of_field)

    matrix = []
    #1. Define Matrix with respect to degree of polynomial
    #   Evaluate the polynomial a_nx^n + ... + d = f(w**d) where d is 0-3rd power
    #.  
    for i in root:
        row = []
        #print(i)
        for j in range(0,len(root)):
            row.append((i)**j%order_of_field)
        matrix.append(row)
    #print(matrix)
    matrix_in_field = GF(matrix)
    inverse = np.linalg.inv(matrix_in_field)
    #print(np.matmul(inverse,matrix_in_field))
    
    
    #2. multiple inverse matrix times constraints this produces wieghts of polynomial
    constraints_in_field = GF(constraints)
    weights = np.matmul(inverse,constraints_in_field)
    
    
    return galois.Poly(weights[::-1], field=GF) 

# Roots of unity for x^4 =1
def roots_of_unity(deg,order_of_field):
    roots = []
    for i in range(1,order_of_field+1):
        if i**deg % order_of_field == 1:
            roots.append(i)
            
    return roots

def coset(element,order_of_field,subgroup):
    cosets = []
    for i in subgroup:
        cosets.append(element*i%order_of_field)
    return cosets



roots = [1,4,16,13]
k1roots = [2,8,15,9]
k2roots = [3,12,14,5]



# In article, it goes from vector a -> function f_a
fa = interpolate_polynomial(A,17,roots)
fb = interpolate_polynomial(B,17,roots)
fc = interpolate_polynomial(C,17,roots)


fl = interpolate_polynomial(l,17,roots)
fr = interpolate_polynomial(r,17,roots)
fo = interpolate_polynomial(o,17,roots)
fm = interpolate_polynomial(m,17,roots)
fselc =  interpolate_polynomial(c,17,roots)

print('FA: ',fa)
print('FB: ',fb)
print('FC: ',fc)
print('FL: ',fl)
print('FR: ',fr)
print('FO: ',fo)
print('FM: ',fm)
print('Fselc: ',fselc)

# Copy Constraints 

- So far, we are able to represent our gates as a family of polynomials. However these polynomials are not sufficent to determine the relationship between the elements in each gate. 

- Why though? Given our family of polynomials we are to translate them into a matrix and then encode them as polynomials! However notice that while the column vectors of our matrix is encoded, there is no data between these interpolated polynomials that define how these polynomials are related. 

- To show the relationship between these functions we will eventually define a polynomial that proves the relationship is valid.  

\begin{matrix}
Column| & A & B & C\\
Gate 1| & 3 & 3 & 9\\
Gate 2| &4 & 4 & 16\\
Gate 3| &5 & 5 & 25\\
Gate 4| &9 & 16& 25\\
\end{matrix}

In our example, the orginal trace we had the following relationships. These relationships are formally called copy constraints. 

$$a_1 = b_1$$
$$a_2 = b_2$$
$$a_3 = b_3$$
$$a_4 = c_1$$

$$b_1=a_1 $$
$$b_2= a_2$$
$$b_3 = a_3$$
$$c_1 = a_4$$

$$c_1 = a_4$$
$$c_2 = b_3$$
$$c_3 = c_4$$
$$c_4 = c_3$$

Vitalik's [Understanding Plonk](https://research.metastate.dev/plonk-by-hand-part-2-the-proof/) has a nice inuitive example on how we will prove that these constraints are satisfied. To summarize: 'To prove copy constraints that hop between different sets of wires, the "alternate" coordinates would be slices of a permutation across all three sets.'


With respect to the copy constraints the permutated sets defined by $H,aH,bH$ are therefore the following:

- $a$ is represented by  $\sigma_1$

\begin{matrix}
\sigma_1 & 2 \\
\sigma_1 & 8 \\
\sigma_1 & 15\\
\sigma_1 & 3\\
\end{matrix}

- $b$ is represented by $\sigma_2$

\begin{matrix}
\sigma_2 & 1 \\
\sigma_2 & 4 \\
\sigma_2 & 16\\
\sigma_2 & 12\\
\end{matrix}


- $c$ is represented by $\sigma_3$



\begin{matrix}
\sigma_3 & 13 \\
\sigma_3 & 9 \\
\sigma_3 & 5\\
\sigma_3 & 14\\
\end{matrix}


Lets look at some sample calucations using rough calculations below.




## Domain and Range of $f_a, f_b, f_c$

\begin{matrix}
x_{a_1} & 1 & f_a & 3\\
x_{a_2} & 4 & f_a &4\\
x_{a_3} & 16 & f_a &5 \\
x_{a_4} & 13 & f_a &16\\
\end{matrix}

\begin{matrix}
x_{b_1} & 2 & f_b &3\\
x_{b_2} & 8 & f_b &4\\
x_{b_3} & 15 & f_b &5 \\
x_{b_4} & 9 & f_b &25\\
\end{matrix}


\begin{matrix}
x_{c_1} & 3 & f_c &9\\
x_{c_2} & 12 & f_c &16\\
x_{c_3} & 14 & f_c &25 \\
x_{c_4} & 5 & f_c &25\\
\end{matrix}


There exists a polynomial $P$ such that P(x) = P(\sigma(x)) such that the copy constraints of our Trace $C$ is satisfied where $\sigma$ is a permutation.

## Note 

This permutation check is the heart of the PLONK. It requires more detail then currently in these notes. A seperate note will be made explore in explicit detail why exactly this method is sufficent. 


# Delete below 

To show that the constraints are held we will show that $f_a(i)f_b(i) f_c(i) =f_a(\sigma(i))f_b(\sigma(i)) f_c(\sigma(i)) $ holds. 

In [ ]:
# Interpolation of \sigmas
s1 = [2,8,15,3]
s2 = [1,4,16,12]
s3 = [13,9,5,14]

fs1 = interpolate_polynomial(s1,17,roots)
fs2 = interpolate_polynomial(s2,17,roots)
fs3 = interpolate_polynomial(s3,17,roots)


# Review 

- We have constructed the trace of our circuit and our copy constraints. 

- We also have run our trusted setup to produce our SRS

- We have all values to now construct our Proof! 

# Proof Construction Overview

The proof consists of five different components that are a straight foward computation. Clearly, the proof presented here is only for educational purposes.


Our Proof will consists of:

- Round 1 - 
    -  Commitment of Wire Polynomials A,B,C
- Round 2 - 
    -  Commitment of Permutation polynomial Z
- Round 3 -
    - Commitment of Quotient polynomial T
- Round 4 - 
    - Opening Evaluations for a,b,c, $\sigma_1$, $\sigma_2$, $z_\omega$
- Round 5 - 
    - Commitment of Proof Polynomials $W_\zeta$, $W_\zeta\omega $


# Round 1 - Produce Wire Polynomial Commitments

- Generate random blinding scalars $(b_1, . . . , b_9)$ $\in$ $F$
    - In this case it is 7,4,11,12,16,2
- Compute wire polynomials $a(X)$, $b(X)$, $c(X)$ 
- Compute Commitments of wiring polynomial $[a]_1,[b]_1,[c]_1$


In [ ]:

########################
# THIS COMMUTES THE WIRE POLYNOMIAL
# g = weights for polynomial
# z = roots
# f = polynomial to commit 
# order = order of field
# cermony = not needed?
########################



def commitNE(g,z,f,order,cermony):
    
    # for step 1 
    # g is degree 1
    # z is degree 4 
    # f is degree 3
    
    #np.polyadd(np.polymul([7,4],[1,0,0,0,-1]),np.array([1,13,3,3][::-1]))[::-1] % 17
    
    #Note f is kinda of fucked up 
    poly = np.polyadd(np.polymul(g,z),np.array(f)[::-1])[::-1] % 17
    return poly

comA = commitNE([7,4],[1,0,0,0,-1],[1,13,3,3],17,cermony)
comB = commitNE([11,12],[1,0,0,0,-1],[7,3,14,13],17,cermony)
comC = commitNE([16,2],[1,0,0,0,-1],[6,5,11,4],17,cermony)

########################
# THIS COMMUTES THE WIRE POLYNOMIAL and COMMITS IT! 
# s = Point
# g = weights for polynomial
# z = roots
# f = interpolated polynomial of vector column
# order = order of field
# cermony = not needed?
########################

def commit(s,g,z,f,order,cermony):
    
    # for step 1 
    # g is degree 1
    # z is degree 4 
    # f is degree 3
    
    #np.polyadd(np.polymul([7,4],[1,0,0,0,-1]),np.array([1,13,3,3][::-1]))[::-1] % 17
    
    #Note f is kinda of fucked up 
    poly = np.polyadd(np.polymul(g,z),np.array(f)[::-1])[::-1] % 17
    #print('daad',poly)

    
    hold=[]
    count = 0 
    cer =cermony[::-1]
    # Get the p_n*s**n where p_n is poly coefficentes and S is the secret group element
    while count < len(poly):
        #
        hold.append(addition(poly[count],cermony[count]))
        count = count +1
    
    p = hold[0]
    # Get sum of all
    for j in range(1,len(hold)):
        #print(p,hold[j])
        p = ec_add(p,hold[j])
    
    #print(hold)
    #print(p)
    return p

    
## Step 1 - Complete
CA = commit(Point(1,2),[7,4],[1,0,0,0,-1],fa.coeffs[::-1],17,cermony)
CB = commit(Point(1,2),[11,12],[1,0,0,0,-1],fb.coeffs[::-1],17,cermony)
CC = commit(Point(1,2),[16,2],[1,0,0,0,-1],fc.coeffs[::-1],17,cermony)
print(CA)
print(CB)
print(CC)

# Round 2 - Compute Polynomial P 

- Generate random blinding scalars $(b_7, . . . , b_9)$ $\in$ $F$
    - In this case it is 14,11,7
- Compute permutation challenges $(\beta, \gamma) \in F$ (in our case these will be random values)
    - beta is 12, gamma is 13
- Compute permutation polynomial $z$
- Commit polynomial $z$

In [ ]:
beta = 12 
gamma = 13

# fA = Column Vector in Circuit
# fB = Column Vector in Circuit
# fC = Column Vector in Circuit
# beta = challenge value provided by verifer
# gamma = challenge value  provided by verifer
# fs1-3 = Permutation Functions
# roots = values to check
# k1,k2 = conjugates

def accumulator(fA,fB,fC,beta,gamma,fs1,fs2,fs3,roots,k1,k2,fieldorder):
    acc = 1 
    hold = []
    for i in range(0,len(fA)-1):
    
    
        a1 = (int(fA[i])+beta*roots[i]+gamma)%fieldorder
        a2 = (int(fB[i])+beta*k1*roots[i]+gamma)%fieldorder
        a3 = (int(fC[i])+beta*k2*roots[i]+gamma)%fieldorder
        aT = (a1*a2*a3)%fieldorder
    
    
    
        b1 = (int(fA[i])+beta*np.polyval(fs1[::-1],roots[i])+gamma)%fieldorder
        b2 = (int(fB[i])+beta*np.polyval(fs2[::-1],roots[i])+gamma)%fieldorder
        b3 = (int(fC[i])+beta*np.polyval(fs3[::-1],roots[i])+gamma)%fieldorder
        bT = (b1*b2*b3)%fieldorder
        
        hold.append((int(pow(int(bT), -1, 17))*aT)%fieldorder)
        
 
    
    things = [1]
    place = 1
    for i in hold:
        place = (i*place)%fieldorder
        things.append(place)
    #print(things)
    return things
    

# The values of acculmulator are found for the roots (domain) [1, 4, 16, 13]
acc = accumulator(A,B,C,beta,gamma,fs1.coeffs[::-1],fs2.coeffs[::-1],fs3.coeffs[::-1],roots,2,3,17)
Accumlator = interpolate_polynomial(acc,17,roots)
Accumlator



#z 
Za=commit(Point(1,2),[14,11,7],[1,0,0,0,-1],Accumlator.coeffs[::-1],17,cermony)


# Round 3
- Compute quotient challenge $\alpha \in F$
    - alpha = 15
- Compute quotient polynomial t(X) 
- Commit  quotient polynomial t(X) into 3 smaller commitments


In [ ]:
alpha = 15
k1 = 2
k2 = 3
comA = commitNE([7,4],[1,0,0,0,-1],[1,13,3,3],17,cermony)
comB = commitNE([11,12],[1,0,0,0,-1],[7,3,14,13],17,cermony)
comC = commitNE([16,2],[1,0,0,0,-1],[6,5,11,4],17,cermony)
L1 = interpolate_polynomial([1,0,0,0],17,roots)




# Big Picture compute t by:
# Find t_1,t_2,t_3,t_4 where t_1 + ... + t_4 * Z*4 -1= \psi
#


# Messy implementation by Computation but it's okay. Its only notes. 


# a + beta x + Gamma
first = np.polyadd(comA[::-1],[gamma,beta][::-1]) %17
last = np.polymul(first,alpha) %17
t21=last[::-1]
# end of a + beta x + Gamma


# b +beta k x + gamma
firstb = np.polyadd(comB[::-1],[gamma,beta*k1][::-1]) %17
t22=firstb[::-1]
# end of b +beta k x + gamma




# c + beta k2 x +gamma
firstc = np.polyadd(comC[::-1],[gamma,beta*k2][::-1]) %17
t23= firstc[::-1]
#end of c + beta k2 x +gamma


#z 
Z=commitNE([14,11,7],[1,0,0,0,-1],Accumlator.coeffs[::-1],17,cermony)



# a + beta Fs1(x) + Gamma
inter = np.polymul(beta,fs1.coeffs[::-1]) % 17
inter2= np.polyadd(inter[::-1],[gamma]) % 17

first = np.polyadd(comA[::-1],inter2) %17
t31 = np.polymul(first,alpha)[::-1] %17
#t31[::-1]
#end of a + beta Fs1(x) + Gamma


# b + beta Fs2(x) + Gamma

interb = np.polymul(beta,fs2.coeffs[::-1]) % 17
inter2b= np.polyadd(interb[::-1],[gamma]) % 17

t32 = np.polyadd(comB[::-1],inter2b)[::-1] %17
#t32[::-1]
#end of b + beta Fs2(x) + Gamma


# c + beta Fs3(x) + Gamma

interc = np.polymul(beta,fs3.coeffs[::-1]) % 17
inter2c= np.polyadd(interc[::-1],[gamma]) % 17

t33 = np.polyadd(comC[::-1],inter2c)[::-1]%17
# end of c + beta Fs3(x) + Gamma

t222 = np.polymul(t21[::-1],t22[::-1])[::-1] % 17
#t222

t233 = np.polymul(t222[::-1],t23[::-1])[::-1] % 17
#t233

t2 = np.polymul(t233[::-1],Z[::-1])[::-1] % 17

# Compute 4th component of quotient

diff = np.polyadd(Z[::-1],[-1][::-1])[::-1]
adiff= np.polymul(diff,alpha*alpha) %17
t4= np.polymul(adiff[::-1],L1.coeffs[::-1])[::-1]%17

# Compute 3rd component of quotient

at = np.polymul(t31[::-1],t32[::-1])[::-1] % 17
att = np.polymul(at[::-1],t33[::-1])[::-1] % 17
ok = np.polymul(att[::-1],[10,3,9,12,7,10,3][::-1])[::-1] % 17
t3=-ok %17

# t_1 disappered and I am not recomputing this

t1 = [1,0,5,4,16,3,3,5,10,5,9,8,7,9]

tt=np.polyadd(t1[::-1],t2[::-1])[::-1] %17
ttt=np.polyadd(t3[::-1],t4[::-1])[::-1]%17
tttt=np.polyadd(tt[::-1],ttt[::-1])[::-1]%17


GF256 = galois.GF(17)

tup= galois.Poly.Degrees([x for x in range(0,22)][::-1],tttt[::-1],field=GF256)
qdown= galois.Poly.Degrees([4,0],[1,-1],field=GF256)
#qdown

finalT= tup//qdown

t = list(np.array(finalT.coeffs))[::-1]


In [ ]:
# Now Split into 3 smaller pieces and commit them
tlow = t[0:6]
tlow_poly = galois.Poly.Degrees([0,1,2,3,4,5],tlow,field = GF256)
tmid = t[6:12]
tmid_poly = galois.Poly.Degrees([0,1,2,3,4,5],tmid,field = GF256)

thigh = t[12:]
thigh_poly = galois.Poly.Degrees([0,1,2,3,4,5],thigh,field = GF256)



def evaluate_Poly_in_EC(p,poly,cermony):
    hold=[]
    count = 0 
    cer = cermony[::-1]
    #print(cer)
    # Get the p_n*s**n where p_n is poly coefficentes and S is the secret group element
    while count < len(poly):
        #
        if poly[count] != 0:
            hold.append(addition(poly[count],cermony[count]))
            count = count +1

        else:
            count = count +1

            continue
        
    
    p = hold[0]
    #print('fafa',p)
    # Get sum of all
    for j in range(1,len(hold)):
        #print(p,hold[j])
        p = ec_add(p,hold[j])
    
    #print('hold',hold)
    #print(p)
    return p



# This is wrong (I think). The commitment and quotient polynomial is a long computation.
# At the end the commitment should just occur at evaluating at S. 
#

CL = evaluate_Poly_in_EC(Point(1,2),tlow,cermony)
CM = evaluate_Poly_in_EC(Point(1,2),tmid,cermony)
CH = evaluate_Poly_in_EC(Point(1,2),thigh,cermony)


# Round 4
- Compute evaluation challenge $\zeta \in F$ 

- Compute opening  evaluations and output them:
 - $\bar{a} = a(\zeta)$,
 - $\bar{b} = b(\zeta)$, 
 - $\bar{c}¯ = c(\zeta)$,
 - $\bar{s}_{\sigma1}= s_{\sigma1}(\zeta)$
 - $\bar{s}_{\sigma2}= s_{\sigma2}(\zeta)$
 - $\bar{Z}_\omega= Z(\zeta*\omega)$
 




In [ ]:
zeta = 5 
cofT = list(np.array(finalT.coeffs))

abar = np.polyval(comA[::-1],5) %17
bbar = np.polyval(comB[::-1],5) %17
cbar = np.polyval(comC[::-1],5) %17
s1bar = np.polyval(fs1.coeffs,5) %17
s2bar = np.polyval(fs2.coeffs,5) %17
tbar = np.polyval(cofT,5) %17 
zwbar = np.polyval([10,3,9,12,7,10,3][::-1],5) %17


print('a',abar)
print('b',bbar)
print('c',cbar)
print('s1',s1bar)
print('s2',s2bar)
print('t',tbar)
print('zw',zwbar)

In [ ]:

# NOT DYNAMIC
def construct_linearization_polynomial(abar,bbar,cbar,s1bar,s2bar,zwbar,fm,fl,fr,fo,beta,gamma,z):
    
    # First Summand
    #a b qm
    abqm = GF256((abar*bbar)%17)*galois.Poly.Degrees([0,1,2,3],fm,field = GF256)
    #a ql
    aql = GF256(abar) * galois.Poly.Degrees([0,1,2,3],fl,field = GF256)
    #b qr
    bqr = GF256(bbar) * galois.Poly.Degrees([0,1,2,3],fr,field = GF256)
    #c q0
    
    # BROKEN HERE
    cq0 = GF256(cbar)* galois.Poly.Degrees([0],fo,field = GF256)
    #qc
    
    t1 = abqm + aql + bqr + cq0
    
    #print('oko',t1)
    
    # Second Summand
    #------------------

    #p1
    

    #abar_add = galois.Poly.Degrees([0],[abar],field=GF256)
    #ads = galois.Poly.Degrees([0,1],[gamma,(5*beta)%17],field=GF256)
    #p1 = abar_add + ads
    
    p1 = (abar + 5*beta + gamma)%17
    #print('p1',GF256((abar + 5*beta + gamma)%17))
    #p2

    #bbar_add = galois.Poly.Degrees([0],[bbar],field=GF256)
    #bds = galois.Poly.Degrees([0,1],[gamma,(5*k1*beta)%17],field=GF256)
    
    p2 = (bbar + 5*beta*k1 + gamma)%17
    
    #p3
    
    #cbar_add = galois.Poly.Degrees([0],[cbar],field=GF256)
    #cds = galois.Poly.Degrees([0,1],[gamma,(5*k2*beta)%17],field=GF256)
    #p3 = cbar_add + cds
    
    p3 = (cbar + 5*beta*k2 + gamma)%17

    
    Z0 = galois.Poly.Degrees([0,1,2,3,4,5,6],Z,field = GF256) 
    
    t1c = (p1 * p2 *p3*alpha) % 17
    #print(t1c,p1,p2,p3,alpha)
    #print(Z)
    t11 = Z0 * GF256(t1c)
    #print(t11)
    
    
    # third Summand
    #------------------
    #p1
    
    p1 = (abar + beta*s1bar + gamma)%17

    
    p2 = (bbar + beta*s2bar + gamma)%17
    

    p3 = ( beta*alpha*zwbar)%17

    
    Z0 = fs3
    
    t1c = (p1 * p2 *p3) % 17
    #print(t1c,p1,p2,p3,alpha)
    #print(Z)
    t2 = Z0 * GF256(t1c)
    #print(t2)
    
    
    # fourth Summand
    
    Z0 = galois.Poly.Degrees([0,1,2,3,4,5,6],Z,field = GF256) 

    t3   = GF256((np.polyval(L1.coeffs[::-1],5) %17)) * Z0 * (alpha*alpha)
    #print(t3)
    
    #
    #print('t',t11+t1+t3)
    return t11+t1+t3
    
    
r= construct_linearization_polynomial(abar,bbar,cbar,s1bar,s2bar,zwbar,fm.coeffs[::-1],fl.coeffs[::-1],fr.coeffs[::-1],fo.coeffs[::-1],beta,gamma,Z)
rbar=r(5)

# Round 5.

- Compute opening proof polynomial $W_\omega$, $W_{\zeta *\omega}$
- Compute commitment of $W_\omega$, $W_{\zeta *\omega}$




In [ ]:
zeta = GF256(5)
randV = GF256(12)
omega = GF256(4)



# Literal Computation of 
rcomp = randV*(r - rbar)
#rcomp = randV*(r)
#print(rcomp,'CHECk')
acomp =(randV**2)*(galois.Poly.Degrees([0,1,2,3,4,5],comA,field = GF256) - GF256(abar))
#print(acomp,'CHECKING')
bcomp =(randV**3)*(galois.Poly.Degrees([0,1,2,3,4,5],comB,field = GF256) - GF256(bbar))
ccomp =(randV**4)*(galois.Poly.Degrees([0,1,2,3,4,5],comC,field = GF256) - GF256(cbar))
s1comp = (randV**5)*(galois.Poly.Degrees([0,1,2,3],fs1.coeffs[::-1],field = GF256)-GF256(s1bar))
#print(s1comp,'CHECKING')

s2comp = (randV**6)*(galois.Poly.Degrees([0,1,2,3],fs2.coeffs[::-1],field = GF256)-GF256(s2bar))

tmid_poly_times = tmid_poly*(zeta**6)
thigh_poly_times = thigh_poly*(zeta**12)
#print(thigh_poly_times,'new',thigh_poly,(4*11) %17)

thefirst =(tlow_poly+tmid_poly_times+thigh_poly_times) - GF256(tbar)


w3 = rcomp + acomp + bcomp +ccomp+s1comp+s2comp + thefirst


# Here is First proof polynomial
womega = w3// galois.Poly.Degrees([0,1],[-zeta,1],field = GF256)
commit_womega = evaluate_Poly_in_EC(Point(1,2),womega.coeffs[::-1],cermony)

In [ ]:
thez = galois.Poly.Degrees([0,1,2],[14,11,7][::-1],field = GF256) * galois.Poly.Degrees([0,1,2,3,4],[1,0,0,0,-1][::-1],field = GF256)+Accumlator
top = thez - GF256(zwbar)
bottom =  galois.Poly.Degrees([0,1],[- zeta*omega,1],field = GF256)
w_zetaomega = top //bottom
commit_w_zetaomega = evaluate_Poly_in_EC(Point(1,2),w_zetaomega.coeffs[::-1],cermony)
commit_w_zetaomega

# Proof Completion

Success: The prover has generated all the values for our proof! Please see below:

In [ ]:
proof = [CA,CB,CC,Za,CL,CM,CH,commit_womega,commit_w_zetaomega,GF256(abar),GF256(bbar),GF256(cbar),GF256(s1bar),GF256(s2bar),rbar,GF256(zwbar)]
proof

# Excerpt from Plonk Paper

See the Plonk paper https://eprint.iacr.org/2019/953. Here follow some meditations on chapter 5 and appendix A.

Plonk uses the fact that polynomials can be uniquely represented as a vector of coefficients and as a multiset of irreducible factors to efficiently proof that one vector is a permutation of another. This is done using applications of the Schwartz-Zippel lemma.

IMHO, the two key innovations in Plonk's permutation check are 1) a protocol for multiset equality checking and 2) a way of encoding a permutation check as a multiset equality check over pairs.

Lemma A.2. (Schwartz-Zippel). Let P(X)∈F 
p
​
 [X 
n
 ] be non-zero and α∈S⊆F 
p
n
​
  uniform random, then

$$ \Pr\left[P(\vec α) = 0 \right] ≤ \frac{\deg P}{\norm S} $$

Corrolary. Given P(X)∈F 
p
​
 [X] with degP≪p and uniform random α∈F 
p
​
 , then P(α)=0 with non-neglibile probability only when P=0.

Corrolary. (Polynomial identity). Given P(X),Q(X)∈F 
p
​
 [X] with degP,degQ≪p and uniform random α∈F 
p
​
 , then P(α)=Q(α) with non-neglibile probability only when P=Q.

Lemma A.3. (Multiset equality) Given two multisets $\set A, \set B$ over $\F_p$ with $\norm{\set A}, \norm{\set B} ≪ p$ and $𝛾 ∈ \F$ uniform random, then $\set A = \set B$ if the following holds with non-neglible probability:

$$ \prod_{a ∈ \set A} \( a + 𝛾 \) = \prod_{b ∈ \set B} \( b + 𝛾 \) $$

Proof. Define $P(X) ≝ \prod_{a ∈ A} (a + X)$ and $Q(X) ≝ \prod_{b ∈ B}(b + X)$. If the above holds with non-neglible probability then by the polynomial identity corrolary we have $P = Q$. From the unique factorization of polynomials it follows that their irreducible factors are the same, which are given in their definition. From this follows $\set A = \set B$. □

Note. We do not require $\norm{\set A} = \norm{\set B}$ and the test will correctly fail when they are of unequal size.

Remark. Compared to lemma A.3 in the paper it is rephrased in terms of multisets and generalized to have multisets of unequal size. The protocol in chapter 5 is readily adjusted to become a multiset-equality protocol.

To do. Rephrased permutation checking protocol as an instance of multiset-equality checking.

To do. Is there utility in constructions where the irreducible factors are of higher degree?

Lemma. (Vector equality). Given two vectors $\vec a, \vec b ∈ \F_p^n$ with $n ≪ p$ and uniform random $\beta \in \F_p$, then $\vec a = \vec b$ if the following holds with non-neglible probability:

$$ \sum_{i ∈ [n]} a_i ⋅ 𝛽^i = \sum_{i ∈ [n]} b_i ⋅ 𝛽^i $$

Proof. Define $P(X) ≝ \sum_{i ∈ [n]} a_i ⋅ X^i$ and $Q(X) ≝ \sum_{i ∈ [n]} a_i ⋅ X^i$. If the above holds with non-neglible probability then by the polynomial identity corrolary we have $P = Q$. From the uniqueness of the coefficient representation of polynomials it follows that $\forall_{i \in [n]} a_i = b_i$ and therefore $\vec a = \vec b$. □

Remark. If $\vec a$ and $\vec b$ have unequal size, the test can incorrectly pass if the final entries of the longer vector are zero, otherwise it should work correctly. The stated form requires them to be equal length. (to do: we could add a $X^{n+1}$ term to bind the length of the vector.)

Note. In the permutation check we use the vector equality check with $n=2$, i.e. pairs.

To do. (Efficient composability). Show that we can do a multiset check over vectors with only two random variables.

Permutation check
Given $\vec a, \vec b ∈ \F_p^n$ and a permutation $𝜎: [n] → [n]$. We want to show that $b_i = a_{𝜎(i)}$.

Take $\vec c$ with distinct values (i.e. $c_i = c_j ⇔ i = j$). Compose the multiset and vector equality protocols to prove the following equality:

$$ \begin{Bmatrix} (a_0, c_0) \\ (a_1, c_1) \\ (a_2, c_2) \\ ⋮ \end{Bmatrix} = \begin{Bmatrix} (b_0, c_{𝜎(0)}) \\ (b_1, c_{𝜎(1)}) \\ (b_2, c_{𝜎(2)}) \\ ⋮ \end{Bmatrix} $$

Proof. Since the elements $c_i$ are unique, the pairs are unique and the multisets are just regular sets. Two sets are equal iff their elements can be brought in correspondance. Enumerate the elements as above, we are looking for correspondances between elements $(a_i, c_i) ≟ (b_j, c_{𝜎(j)})$. The second element in the pair is equal only if $i = 𝜎(j)$, this means a correspondance exists iff $b_j = a_{𝜎(j)}$. □

Claim A.1. If the following holds with non-negligible probability over random $𝛽, 𝛾 \in \mathbb F$

$$ ∏_i \( a_i + 𝛽 ⋅ i + 𝛾 \) = ∏_i \( b_i + 𝛽 ⋅ 𝜎\(i\) + 𝛾 \) $$

then $∀_{i \in [n]} b_i = a_{𝜎(i)}$.

To do. Demostrate this follows concretely from the above.

